# 4일차 6교시 Spark JDBC to MySQL

### 목차
* 1. Spark JDBC 환경구성
* 2. MySQL 접속 예제
* 4. References
  * https://spark.apache.org/docs/2.4.3/sql-data-sources-jdbc.html
  

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Data Engineer Intermediate Day4") \
    .config("spark.dataengineer.intermediate.day4", "tutorial-6") \
    .getOrCreate()
spark.version

'2.4.5'

### 1. Spark JDBC 환경구성

#### 1.1. MySQL 기동
```bash
export PROJECT_HOME=`pwd`
docker run --name mysql --network=day3_default -e MYSQL_ROOT_PASSWORD=rootpass -e MYSQL_DATABASE=testdb -e MYSQL_USER=user -e MYSQL_PASSWORD=pass -p 3306:3306 -v $PROJECT_HOME/custom:/etc/mysql/conf.d -d psyoblade/data-engineer-intermediate-day1-mysql
```

#### 1.2. SparkSession 으로부터 SQLContext 생성
```scala
sc = spark.sparkContext
sqlContext = new org.apache.spark.sql.SQLContext(sc)
```

#### 1.3. Spark Notebook 네트워크를 MySQL 으로 연결
```bash
docker network connect day3_default mysql
```

### 2. MySQL 접속 예제

In [2]:
from pyspark.sql.context import SQLContext
sc = spark.sparkContext
sqlContext = SQLContext(sc)
seoul_popular_trip = sqlContext.read.format("jdbc") \
.option("url", "jdbc:mysql://mysql:3306/testdb") \
.option("driver", "com.mysql.jdbc.Driver") \
.option("dbtable", "seoul_popular_trip") \
.option("user", "user") \
.option("password", "pass") \
.load()

In [3]:
seoul_popular_trip.printSchema()
seoul_popular_trip.show(5)

root
 |-- category: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- naddress: string (nullable = true)
 |-- tel: string (nullable = true)
 |-- tag: string (nullable = true)

+--------+---+----------------------------+---------------------------+-----------------------------+------------+-----------------------------------+
|category| id|                        name|                    address|                     naddress|         tel|                                tag|
+--------+---+----------------------------+---------------------------+-----------------------------+------------+-----------------------------------+
|       0|281|                    통인시장|110-043 서울 종로구 통인...|03036 서울 종로구 자하문로...| 02-722-0911|엽전도시락,종로통인시장,통인시장...|
|       0|345|                      타르틴|140-863 서울 용산구 이태...|04350 서울 용산구 이태원로...|02-3785-3400| 타르틴,이태원디저트카페,파이,런...|
|       0|383|                        해랑|135-8

In [4]:
from pyspark.sql.functions import *
# seoul_popular_trip.limit(5).select("id", "name", explode(split("tag", ","))).groupBy("name").count().show()
top10 = seoul_popular_trip.select("id", "name", explode(split("tag", ","))) \
.groupBy("name").count() \
.orderBy(desc("count"))

In [5]:
top10.show()

+-------------------------------+-----+
|                           name|count|
+-------------------------------+-----+
|                  CJ 올리브마켓|   36|
|       더 스트리트 (THE STREET)|   32|
|                       통인시장|   30|
|          신세계면세점 (명동점)|   28|
|              신세계백화점 본점|   27|
|                     401 (홍대)|   26|
|젠틀몬스터 신사 플래그쉽 스토어|   26|
|                     장진우식당|   26|
|                 설빙 신림2호점|   24|
|               스튜디오콘크리트|   22|
|                아티지 (ARTAGE)|   22|
|                     두레한식당|   22|
|      너드온어스 (Nerd on ea...|   22|
|         리아네이처 (Lyanature)|   22|
|               샤오짠(Xiaozhan)|   22|
|                롯데면세점 본점|   21|
|      펭귄스카페 (S&Penguins...|   20|
|             신라아이파크면세점|   20|
|               명동지하쇼핑센터|   19|
|               종각지하쇼핑센터|   19|
+-------------------------------+-----+
only showing top 20 rows



In [6]:
top10.write.format("jdbc") \
.option("url", "jdbc:mysql://mysql:3306/testdb") \
.option("driver", "com.mysql.jdbc.Driver") \
.option("dbtable", "top10") \
.option("user", "user") \
.option("password", "pass") \
.save()

#### 2.1. 저장된 데이터 확인
```bash
docker exec -it mysql mysql -uuser -p
mysql> use testdb;
mysql> select * from top10 order by count desc limit 10;
```